In [ ]:
!pip install torch torchvision torchaudio
!pip install transformers pillow matplotlib
import torch
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from transformers import CLIPProcessor, CLIPModel
from google.colab import files
print("Libraries installed and imported!")

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
print("Model loaded successfully!")

uploaded = files.upload()

Libraries installed and imported!


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Model loaded successfully!


Saving img01_pomeranian.jpg to img01_pomeranian.jpg
Saving img02_bichon.jpg to img02_bichon.jpg
Saving img03_dessert.jpg to img03_dessert.jpg
Saving img04_bouquet.jpg to img04_bouquet.jpg
Saving img05_snowy.jpg to img05_snowy.jpg
Saving img06_sunny.jpg to img06_sunny.jpg
Saving img07_tsukemen.jpg to img07_tsukemen.jpg
Saving img08_cardholder.jpg to img08_cardholder.jpg
Saving img09_rose.jpg.jpg to img09_rose.jpg.jpg
Saving img10_airplane.jpg to img10_airplane.jpg


In [ ]:
image_files = list(uploaded.keys())
image_files

['img01_pomeranian.jpg',
 'img02_bichon.jpg',
 'img03_dessert.jpg',
 'img04_bouquet.jpg',
 'img05_snowy.jpg',
 'img06_sunny.jpg',
 'img07_tsukemen.jpg',
 'img08_cardholder.jpg',
 'img09_rose.jpg.jpg',
 'img10_airplane.jpg']

In [ ]:
texts_detailed = [
    "a small white Pomeranian walking at night on the street",
    "two fluffy white Bichon Frises sitting together in a stroller",
    "a phone screen showing assorted desserts on white plates",
    "a bouquet of pastel flowers including roses and hydrangeas",
    "a snowy empty street at night with street lamps",
    "a sunny street lined with power lines and green trees",
    "Japanese tsukemen noodles served with broth, pork, egg and seaweed",
    "a blue shopping bag and a wrapped gift box with a black leather cardholder beside them",
    "a hand holding a wrapped red rose in a subway station",
    "an airplane wing and terminal seen through a plane window at sunset",
]
texts = texts_detailed

In [ ]:
texts_simple = [
    "a white dog at night",
    "two white dogs in a stroller",
    "desserts on plates",
    "a bouquet of flowers",
    "a snowy street",
    "a sunny street with trees",
    "a bowl of noodles",
    "a gift box and a cardholder",
    "a hand holding a rose",
    "a plane wing at an airport",
]
texts = texts_simple

In [ ]:
texts_abstract = [
    "a quiet walk in the darkness",                         #img01_pomeranian
    "companionship and comfort while being moved",          #img02_bichon
    "a sweet indulgence and enjoyment",                     #img03_dessert
    "a gentle expression of affection and beauty",          #img04_bouquet
    "tranquility in a cold silent night",                   #img05_snowy
    "a warm peaceful outdoor afternoon",                    #img06_sunny
    "a savory moment of comfort and warmth",                #img07_tsukemen
    "a thoughtful gesture of gifting and care",             #img08_cardholder
    "a symbol of affection in a fleeting moment",           #img09_rose
    "a feeling of waiting before a long journey",           #img10_airplane
]
texts = texts_abstract

In [ ]:
# Load and preprocess images
images = [Image.open(f).convert("RGB") for f in image_files]

inputs = processor(text=texts, images=images, return_tensors="pt", padding=True)

with torch.no_grad():
    outputs = model(**inputs)

# Separate embeddings
image_embeds = outputs.image_embeds
text_embeds = outputs.text_embeds

# Normalize embeddings (cosine similarity)
normalized_image_embeds = torch.nn.functional.normalize(image_embeds, p=2, dim=-1)
normalized_text_embeds = torch.nn.functional.normalize(text_embeds, p=2, dim=-1)

# Compute similarity matrix (10x10)
similarities = normalized_text_embeds @ normalized_image_embeds.T

# Print similarity matrix
print("Similarity matrix:\n")
for i in range(10):
  for j in range(10):
    print(similarities[i][j].item(), end=" ")
  print("\n")
print(similarities)

Similarity matrix:

0.3104722499847412 0.2723226249217987 0.11799539625644684 0.118074432015419 0.20141997933387756 0.13340887427330017 0.057292282581329346 0.13115128874778748 0.14294657111167908 0.13610795140266418 

0.24671003222465515 0.3625601828098297 0.13836142420768738 0.14876309037208557 0.13189539313316345 0.1707851141691208 0.11479255557060242 0.1335066556930542 0.125996932387352 0.17329654097557068 

0.15399208664894104 0.16986817121505737 0.29125136137008667 0.1889285296201706 0.13801375031471252 0.13564260303974152 0.20334148406982422 0.15265101194381714 0.10907433927059174 0.15043893456459045 

0.1206083595752716 0.11944325268268585 0.11463885009288788 0.31591445207595825 0.04735203832387924 0.11029333621263504 0.12935873866081238 0.1506100296974182 0.15668466687202454 0.08631354570388794 

0.12423253059387207 0.13580666482448578 0.08362963050603867 0.0857703685760498 0.2630811333656311 0.14202068746089935 0.06262949854135513 0.07642120122909546 0.11224684119224548 0.113

In [ ]:
print("\n=== Ranking for each text query ===\n")

for i, text in enumerate(texts):
    sims = similarities[i]
    ranking = torch.argsort(sims, descending=True)
    print(f"\nText {i+1}: {text}")
    for r in ranking[:5]:
        print(f"   Image {r.item()+1} ({image_files[r.item()]})  --> similarity {sims[r]:.4f}")



=== Ranking for each text query ===


Text 1: a small white Pomeranian walking at night on the street
   Image 1 (img01_pomeranian.jpg)  --> similarity 0.3105
   Image 2 (img02_bichon.jpg)  --> similarity 0.2723
   Image 5 (img05_snowy.jpg)  --> similarity 0.2014
   Image 9 (img09_rose.jpg.jpg)  --> similarity 0.1429
   Image 10 (img10_airplane.jpg)  --> similarity 0.1361

Text 2: two fluffy white Bichon Frises sitting together in a stroller
   Image 2 (img02_bichon.jpg)  --> similarity 0.3626
   Image 1 (img01_pomeranian.jpg)  --> similarity 0.2467
   Image 10 (img10_airplane.jpg)  --> similarity 0.1733
   Image 6 (img06_sunny.jpg)  --> similarity 0.1708
   Image 4 (img04_bouquet.jpg)  --> similarity 0.1488

Text 3: a phone screen showing assorted desserts on white plates
   Image 3 (img03_dessert.jpg)  --> similarity 0.2913
   Image 7 (img07_tsukemen.jpg)  --> similarity 0.2033
   Image 4 (img04_bouquet.jpg)  --> similarity 0.1889
   Image 2 (img02_bichon.jpg)  --> similarity 0.1699